In [1]:
import pandas as pd

In [2]:
import tensorflow as tf
print(tf.__version__)

2.20.0


In [3]:
cars_cleaned = pd.read_csv('../../data/scrapped/voitures_clean.csv')
cars_cleaned.head()

,Ville,Marque,Modèle,Année-Modèle,Kilométrage,Type de carburant,Puissance fiscale,Boite de vitesses,Nombre de portes,Origine,Première main,Prix,Ville_grouped,Age_vehicule
0,Temara,Fiat,Punto,2007,224999.5,Diesel,5.0,Manuelle,5.0,WW au Maroc,Non,60000.0,Temara,18
1,Casablanca,Volkswagen,Touareg,2005,2499.5,Diesel,10.0,Automatique,5.0,WW au Maroc,Oui,90000.0,Casablanca,20
2,Dakhla,Toyota,Prado,2007,224999.5,Diesel,12.0,Manuelle,5.0,WW au Maroc,Non,97000.0,Autres,18
3,Khouribga,Volkswagen,Tiguan,2014,184999.5,Diesel,8.0,Automatique,5.0,Dédouanée,Non,255000.0,Autres,11
4,Meknès,Peugeot,308,2009,224999.5,Diesel,6.0,Manuelle,5.0,WW au Maroc,Non,85000.0,Meknès,16


In [4]:
df1 = cars_cleaned.copy()

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

df1 = df1.drop(columns=['Ville', 'Année-Modèle'], errors='ignore')
df1['logPrix'] = np.log1p(df1['Prix'])

# Variables / target
X = df1.drop(columns=['Prix', 'logPrix'], errors='ignore')
y = df1['logPrix']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

ohe_cols = [
    'Boite de vitesses',
    'Type de carburant',
    'Origine',
    'Première main',
    'Ville_grouped'
]

X_train = pd.get_dummies(X_train, columns=ohe_cols, drop_first=True)
X_test  = pd.get_dummies(X_test,  columns=ohe_cols, drop_first=True)

X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

for col in ['Marque', 'Modèle']:
    means = X_train.join(y_train).groupby(col)['logPrix'].mean()
    
    X_train[col + '_te'] = X_train[col].map(means)
    
    global_mean = y_train.mean()
    X_test[col + '_te'] = X_test[col].map(means).fillna(global_mean)

X_train = X_train.drop(columns=['Marque', 'Modèle'], errors='ignore')
X_test  = X_test.drop(columns=['Marque', 'Modèle'], errors='ignore')

num_cols = [
    'Kilométrage',
    'Age_vehicule',
    'Nombre de portes',
    'Puissance fiscale',
    'Marque_te',
    'Modèle_te'
]

scaler = StandardScaler()

X_train_scaled = X_train.copy()
X_test_scaled  = X_test.copy()

X_train_scaled[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test_scaled[num_cols]  = scaler.transform(X_test[num_cols])


model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)


history = model.fit(
    X_train_scaled,
    y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    verbose=1
)


test_loss, test_mae = model.evaluate(X_test_scaled, y_test, verbose=0)
print("MAE (log-prix) :", test_mae)


y_pred_log = model.predict(X_test_scaled).ravel()
y_pred = np.expm1(y_pred_log)
y_real = np.expm1(y_test)

mae_real = np.mean(np.abs(y_real - y_pred))
r2 = r2_score(y_real, y_pred)

print("MAE réel (DH) :", mae_real)
print("R² sur prix réels :", r2)

C:\Users\HP830G5\anaconda3\envs\tf_env\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
1090/1090 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - loss: 4.1054 - mae: 0.8723 - val_loss: 0.0918 - val_mae: 0.2210
Epoch 2/50
1090/1090 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.0707 - mae: 0.1957 - val_loss: 0.0630 - val_mae: 0.1807
Epoch 3/50
1090/1090 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.0647 - mae: 0.1876 - val_loss: 0.0594 - val_mae: 0.1732
Epoch 4/50
1090/1090 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 0.0635 - mae: 0.1862 - val_loss: 0.0727 - val_mae: 0.1962
Epoch 5/50
1090/1090 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 0.0621 - mae: 0.1844 - val_loss: 0.0652 - val_mae: 0.1865
Epoch 6/50
1090/1090 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - loss: 0.0615 - mae: 0.1835 - val_loss: 0.0567 - val_mae: 0.1675
Epoch 7/50
1090/1090 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - loss: 0.0625 - mae: 0.1851 - val_loss: 0.0612 - val_mae: 0.1798
Epoch 8/50
1090/1090 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.0596 - mae: 0.1802 - val_loss: 0.0634 - val_mae: 0.1816
Epoch 9/50
1090/1090 ━━━━━━━━━━━━━━━━━

# Change hyperparametres

In [6]:
model = keras.Sequential([
    layers.Dense(256, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0005),
    loss='mse',
    metrics=['mae']
)

early_stop = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    X_train_scaled,
    y_train,
    validation_split=0.2,
    epochs=150,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

test_loss, test_mae = model.evaluate(X_test_scaled, y_test, verbose=0)

y_pred_log = model.predict(X_test_scaled).ravel()
y_pred = np.expm1(y_pred_log)
y_real = np.expm1(y_test)

mae_real = np.mean(np.abs(y_real - y_pred))
r2 = r2_score(y_real, y_pred)

print("MAE réel (DH) :", mae_real)
print("R² sur prix réels :", r2)

Epoch 1/150


C:\Users\HP830G5\anaconda3\envs\tf_env\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1090/1090 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - loss: 5.4938 - mae: 1.4675 - val_loss: 0.3450 - val_mae: 0.4489
Epoch 2/150
1090/1090 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - loss: 0.9974 - mae: 0.7920 - val_loss: 0.2329 - val_mae: 0.4016
Epoch 3/150
1090/1090 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 0.6987 - mae: 0.6639 - val_loss: 0.3403 - val_mae: 0.5267
Epoch 4/150
1090/1090 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 0.5054 - mae: 0.5645 - val_loss: 0.2877 - val_mae: 0.4844
Epoch 5/150
1090/1090 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 0.3312 - mae: 0.4550 - val_loss: 0.1103 - val_mae: 0.2671
Epoch 6/150
1090/1090 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 0.2146 - mae: 0.3639 - val_loss: 0.0753 - val_mae: 0.2031
Epoch 7/150
1090/1090 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 0.1298 - mae: 0.2790 - val_loss: 0.0876 - val_mae: 0.2323
Epoch 8/150
1090/1090 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - loss: 0.0837 - mae: 0.2196 - val_loss: 0.0520 - val_mae: 0.1621
Epoch 9/150
1090/1090 ━━━━━━━━━━━━━━━━━━━━

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))

# LOSS
plt.subplot(1,2,1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.title('Loss ANN')
plt.legend()

# MAE
plt.subplot(1,2,2)
plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Val MAE')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.title('MAE ANN')
plt.legend()

plt.tight_layout()
plt.show()